In [1]:
import os
import math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
from nltk.stem.snowball import PorterStemmer

# Arquivos Pythons com funções e constantes
from qs_constants import AppConstants
from qs_functions import load_qa_pairs, normalize_qa, remove_duplication, remove_bad_answer, get_context_files
from qs_classes import TitleGroup

In [2]:
nlp = spacy.load('en_core_web_lg')
stemmer = PorterStemmer()

In [3]:
def carrega_dataset():
    df_all = load_qa_pairs()
    df_preparado = df_all[(df_all.Question.notna()) &
                          (df_all.Answer.notna())]
    vet_QA = df_preparado[["Question", "Answer", "ArticleTitle","ArticleFile"]].to_numpy()
    vet_QA = normalize_qa(vet_QA, lowercase=False)
    vet_QA = remove_bad_answer(vet_QA)
    vet_QA = remove_duplication(vet_QA)
    return vet_QA


In [4]:
def agrupa_perguntas_por_assunto(vet_QA):
    qa_agrupada_assunto = {}
    for qa in vet_QA:
        assunto = qa[AppConstants.COL_TITULO]
        assunto_preparado = assunto.lower().replace("_", " ")

        if assunto_preparado not in qa_agrupada_assunto:
            grupo = TitleGroup()
            qa_agrupada_assunto[assunto_preparado] = grupo
        else:
            grupo = qa_agrupada_assunto[assunto_preparado]

        grupo.qa_pairs.append(qa[0:2])

        # existem arquivos com nan
        file = qa[AppConstants.COL_CONTEXT_FILE]
        if isinstance(file, str):
            grupo.files.add(file)
    return qa_agrupada_assunto
    
    

In [5]:
def carrega_contexto_assunto(qa_agrupada_assunto):
    cont_files = 0
    for k_grupo in qa_agrupada_assunto:    
        grupo = qa_agrupada_assunto[k_grupo]
        for ctx_file in grupo.files:
            cont_files += 1
            print('                                                            ', end='\r')
            print(k_grupo, ctx_file, ' - ', cont_files, ' processados', end='\r')
            file_path = os.path.join(AppConstants.CONTEXT_TEXT_PATH, ctx_file + AppConstants.CONTEXT_FILE_EXTENSION)
            with open(file_path, 'r', encoding='utf-8') as f_ctx:
                ctx_file_doc = f_ctx.read()
                ctx_nlp_doc = nlp(ctx_file_doc)
                if ctx_nlp_doc.ents:
                    for ent in ctx_nlp_doc.ents:
                        entidade = ent.text
                        grupo.context_ner.add(entidade)



In [14]:
# retorna uma lista de asssuntos que tem potencial match com a pergunta
def get_assunto_pergunta(qa_agrupada_assunto, pergunta):
    doc_pergunta = nlp(pergunta)
    lista_assunto_match = []
    
    for k_assunto in qa_agrupada_assunto:
        grupo = qa_agrupada_assunto[k_assunto]
        achou = False
        como_achou = ''
        
        # NER pergunta
        if doc_pergunta.ents:
            for entidade in doc_pergunta.ents:
                entidade_text = entidade.text.lower()
                for entidade_token in entidade_text.split():
                    if entidade_token in k_assunto:
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA
                        
        # Prcura por nomes
        if not achou and doc_pergunta.noun_chunks:
            for noun in doc_pergunta.noun_chunks:
                for n_part in noun.text.split():
                    if n_part in k_assunto:
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA    
    
         # Lemmanization
        if not achou:
            doc_assunto = nlp(k_assunto)
            for token_assunto in doc_assunto:
                if token_assunto.is_stop:
                    continue
                assunto_lemma = token_assunto.lemma_.lower()
                for token_pergunta in doc_pergunta:
                    if token_pergunta.is_stop:
                        continue
                    if assunto_lemma == token_pergunta.lemma_.lower():
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION
                if not achou:
                    for token_pergunta in doc_pergunta:
                        if token_pergunta.is_stop:
                            continue
                        if stemmer.stem(token_pergunta.text) == stemmer.stem(token_assunto.text):
                            achou = True
                            como_achou = AppConstants.METODO_BUSCA_ASSUNTO_STEMMER    
                    
        # NER context file            
        if not achou:
            if doc_pergunta.ents:
                for entidade in doc_pergunta.ents:
                    entidade_text = entidade.text
                    if entidade_text in grupo.context_ner:
                            achou = True
                            como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO
                            
        
        if achou:
            lista_assunto_match.append((k_assunto, como_achou))
            
    return lista_assunto_match

In [17]:
# dada uma lista de matchs para assunto retonra aquele que teoricamente seria o de maior peso, seguindo a ordem de prioridade por busca
def get_assunto_match_principal(lista_assunto_match):
    
    def encontra_metodo_busca(lista_assunto, metodo_busca_pesquisa):
        for (assunto, metodo_busca) in lista_assunto_match:
            if metodo_busca == metodo_busca_pesquisa:
                return assunto
        return None
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA
        
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_STEMMER)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_STEMMER
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO
    
    

In [18]:
# imprime as perguntas cuja a entidade não conseguiu ser encontrada

vet_QA = carrega_dataset()
qa_agrupada_assunto = agrupa_perguntas_por_assunto(vet_QA)
carrega_contexto_assunto(qa_agrupada_assunto)

total = 0
qtd_achou_lista = 0
qtd_achou_lista_errada = 0
qtd_achou_unico = 0
qtd_achou_unico_lista = 0
qtd_nao_achou = 0
qtd_achou_unico_incorreto = 0


estatistica_busca = {AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA: 0, 
                     AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA : 0,
                     AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION : 0,
                     AppConstants.METODO_BUSCA_ASSUNTO_STEMMER : 0,
                     AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO : 0,
                     'PERGUNTA_SEM_CONTEXTO' : 0
                    }

for k_assunto in qa_agrupada_assunto:
    grupo = qa_agrupada_assunto[k_assunto]
    for qa in grupo.qa_pairs:
        total += 1
        achou = False
        como_achou = ""
        
        pergunta = qa[AppConstants.COL_PERGUNTA]
        
        lista_assunto_match = get_assunto_pergunta(qa_agrupada_assunto, pergunta)
        
        if len(lista_assunto_match) == 1:
            (assunto_encontrado, metodo_busca) = lista_assunto_match[0]
            if assunto_encontrado != k_assunto:
                qtd_achou_unico_incorreto += 1
            else:
                qtd_achou_unico += 1
                estatistica_busca[metodo_busca] += 1
            
        elif len(lista_assunto_match) > 0:
            assunto_encontrado, metodo_busca = get_assunto_match_principal(lista_assunto_match)
            if assunto_encontrado == k_assunto:
                qtd_achou_unico_lista += 1
                estatistica_busca[metodo_busca] += 1
            else:
                achou_lista = False
                for (assunto_encontrado, metodo_busca) in lista_assunto_match:
                    if assunto_encontrado != k_assunto:
                        continue
                    achou_lista = True
                if achou_lista:
                    qtd_achou_lista += 1
                else:
                    qtd_achou_lista_errada += 1
        else:
            print("Não achou ", k_assunto," - ", pergunta)
            qtd_nao_achou +=1
            for token_sem_ctx in AppConstants.TOKENS_COM_FALTA_CONTEXTO:
                if token_sem_ctx in pergunta:
                    estatistica_busca['PERGUNTA_SEM_CONTEXTO'] += 1
        
        
print ('Total:', total)
print('Achou Unico', qtd_achou_unico)
print('Achou Unico Pela Lista através de prioridade entre métodos ', qtd_achou_unico_lista)
print('Achou Unico Incorreto', qtd_achou_unico_incorreto)
print('Achou Lista ', qtd_achou_lista)
print('Achou Lista Errada ', qtd_achou_lista_errada)
print('Não Achou', qtd_nao_achou)
print(estatistica_busca)

Não achou  abraham lincoln  -  Did his mother die of pneumonia?
Não achou  anders celsius  -  What is named after him?
Não achou  anders celsius  -  What is temperature would water have to be to be halfway between its standard boiling and freezing point?
Não achou  beetle  -  Which defense mechanism uses colour or shape to deceive potential enemies?
Não achou  beetle  -  What are prey of various animals including birds and mammals?
Não achou  beetle  -  Oxygen is what?
Não achou  beetle  -  What happened on plants and fungi?
Não achou  canada  -  What have been inhabited for millennia by aboriginal peoples?
Não achou  canada  -  How do cuba and declining participation relate?
Não achou  duck  -  Why are ducklings particularly vulnerable?
Não achou  elephant  -  Do they prefer forested areas?
Não achou  finland  -  Do women live longer than men?
Não achou  gerald ford  -  How do kwajalein and eniwetok relate?
Não achou  \n  -  hard
Não achou  gray wolf  -  What is surplus killing?
Não a

In [25]:
x = nlp('What areas do the Grevy\'s Zebras inhabit?')
for t in x:
    print(t.text, t.lemma_.lower())

What what
areas area
do do
the the
Grevy grevy
's 's
Zebras zebras
inhabit inhabit
? ?


In [21]:
qa_agrupado_assunto['alessandro volta'].context_ner

{'10.000 lira',
 '10000 Italian Lire',
 '1745-1827',
 '1762',
 '1764',
 '1774',
 '1775',
 '1776-77',
 '1779',
 '1791',
 '1794',
 '1796',
 '1796 to 1815',
 '1800',
 '1810',
 '1815',
 '1819',
 '1881',
 '2',
 '2003',
 '2006',
 'A year later',
 'Abraham Bennet',
 'Alberto Gigli Berzolari',
 'Alessando Volta',
 'Alessandro Giuseppe Antonio Anastasio',
 'Alessandro Giuseppe Antonio Anastasio Volta',
 'Alessandro Volta',
 'Alessandro Volta\n*  Alessandro Volta',
 'Austria',
 'Austrian',
 'Camnago',
 'Camnago Volta',
 'Catholic Encyclopedia',
 'Como',
 'Como\n',
 'FRS',
 'February 18, 1745 - March 5, 1827',
 'February 18, 1745 – March 5, 1827',
 'France',
 'French',
 'Galvani',
 'Giovanni',
 'Giuliano Pancaldi',
 'Italian',
 'Italy',
 'Johan Carl Wilcke  ',
 'Johan Wilcke',
 'Lake Como',
 'Latin',
 'Leyden',
 'Lombardy',
 'Ludovico Peregrini',
 'Luigi Galvani',
 'Luigi Galvani\n* Eudiometer\n* History',
 'March 5, 1827',
 'Milan',
 'Napoleon',
 'Nuova',
 'Padua',
 'Pioneer',
 'Princeton Univer

In [26]:
doc = nlp('What method of locomotion do Kangaroos Use')

In [32]:
for n in doc.noun_chunks:
    print(n.lemma_)

what method
locomotion
Kangaroos
